## Choix du cluster

In [1]:
from pandas import read_csv,to_datetime, DataFrame
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import zscore
import os
from statsmodels.tsa.seasonal import seasonal_decompose
from ydata_profiling import ProfileReport
from numpy import array, newaxis
from tslearn.clustering import TimeSeriesKMeans, silhouette_score,KShape, KernelKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.cluster import DBSCAN
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
# Charger les données
AlldfContinent = read_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\Processed\Product families over time.csv")

# Transposer le DataFrame pour obtenir les Product Families comme colonnes
df2 = AlldfContinent.set_index('Product Family').T
df2.index = to_datetime(df2.index)
X = df2.T.values
product_families = df2.columns

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
D:\WK\ydongue\AppData\Local\Temp\ipykernel_15088\421283029.py:2: SyntaxWarning: invalid escape sequence '\A'
  AlldfContinent = read_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\Processed\Product families over time.csv")


In [4]:
series_scaled = zscore(X)
n_clusters = 3  # Nombre de clusters souhaité
mtr = "laplacian"
model = KernelKMeans(n_clusters=n_clusters, kernel=mtr, n_init=150, max_iter=100, random_state=0)
labels = model.fit_predict(series_scaled)

results_dfkernelKmeans = DataFrame({
    'productfamilies': product_families,
    'clusters': labels})

print(results_dfkernelKmeans)

results_dfkernelKmeans.to_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\clustered_product_families.csv", index=False)


   productfamilies  clusters
0                A         1
1                B         1
2                C         1
3                D         1
4                E         2
5                F         2
6                G         0
7                H         0
8                I         0
9                J         1
10               K         0
11               L         0
12               M         0
13               N         0
14               O         0
15               P         0
16               Q         0


<>:13: SyntaxWarning: invalid escape sequence '\A'
<>:13: SyntaxWarning: invalid escape sequence '\A'
D:\WK\ydongue\AppData\Local\Temp\ipykernel_15088\2630724063.py:13: SyntaxWarning: invalid escape sequence '\A'
  results_dfkernelKmeans.to_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\clustered_product_families.csv", index=False)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\tslearn\utils\utils.py:90: UserWarning: 2-Dimensional data passed. Assuming these are 17 1-dimensional timeseries
  warnings.warn(
